In [36]:
import pandas as pd
import numpy as np
import random
import math

import feat_eng as fe
import data_selector_items as dsi
import params_newsvendor as prm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from mip import Model, xsum, minimize, INTEGER, CONTINUOUS, CutType, OptimizationStatus

from qpth.qp import QPFunction

from joblib import Parallel, delayed

from sklearn.preprocessing import MinMaxScaler

In [37]:
is_cuda = False
dev = torch.device('cpu')  
if torch.cuda.is_available():
    is_cuda = True
    dev = torch.device('cuda')  

## 0. Data

In [38]:
# Setting the seeds to allow replication
# Changing the seed might require hyperparameter tuning again
# Because it changes the deterministic parameters
seed_number = 0
np.random.seed(seed_number)
torch.manual_seed(seed_number)
random.seed(seed_number)

In [39]:
# Path of data files
path_data = './data/'

# Read historical data
sales = pd.read_csv(path_data + 'sales_train_evaluation.csv')

# Spliting the data in days
start_tr_day = 500
start_val_day = 1742
start_test_day = 1842
end_day = 1941

# N of items to use (Lower will be much faster)
# In the paper, this is d_z
n_items = 4

# All useful items
sku_ids = dsi.select_items(sales, start_tr_day)

# Sample only n_items to use
random.seed(seed_number)
sku_ids = random.sample(sku_ids, n_items)
sku_ids = list(set(sku_ids))
n_items = len(sku_ids)


# Build training and test from historical data
data_train, data_val, data_test, feat, n_items = fe.build_data(
    path_data, sales, sku_ids, 
    start_tr_day, start_val_day, start_test_day, end_day)

data_train.fillna(0, inplace=True)
data_val.fillna(0, inplace=True)
data_test.fillna(0, inplace=True)

dx = len(feat)

# Number of batch_size samples in the SGDs
batch_size = 32 # Number of days for combined approaches

# Here we change a bit the test data in order to increase the integrality gap 
# of the optimization problem (didactic purpose). Otherwise the Continuous and 
# Discrete version would have approximately the same results.
data_train['qty'] = data_train['qty']*np.random.normal(1, 0.07)
data_val['qty'] = data_val['qty']*np.random.normal(1, 0.07)
data_test['qty'] = data_test['qty']*np.random.normal(1, 0.07)

scaler = MinMaxScaler()
scaler.fit(data_train[feat])

data_train.loc[:, feat] = scaler.transform(data_train[feat])
data_val.loc[:, feat] = scaler.transform(data_val[feat])
data_test.loc[:, feat] = scaler.transform(data_test[feat])
        
X_train = torch.tensor(np.array(data_train[feat]).astype('float32'), requires_grad= True, device=dev)
y_train = torch.tensor(np.array(data_train['qty']).astype('float32'), requires_grad= True, device=dev)

X_val = torch.tensor(np.array(data_val[feat]).astype('float32'), requires_grad= True, device=dev)
y_val = torch.tensor(np.array(data_val['qty']).astype('float32'), requires_grad= True, device=dev)

X_test = torch.tensor(np.array(data_test[feat]).astype('float32'), requires_grad= True, device=dev)
y_test = torch.tensor(np.array(data_test['qty']).astype('float32'), requires_grad= True, device=dev)

In [40]:
def generate_batches(data):
    batches_idx = []
    n_batches = int(np.floor(data['d'].nunique() / batch_size))
    for i in range(0, n_batches):
        days = data['d'].unique()
        idx = data[data['d'].isin(
            np.random.choice(days, batch_size, replace=False))].index.tolist()
        if len(idx) == n_items*batch_size:
            batches_idx.append(idx)
    return batches_idx

In [41]:
NUM_BATCHES = len(generate_batches(data_train))

In [42]:
params_t, params_np = prm.get_params(n_items, is_discrete=False, 
                                     q_factor = 0.01, # Quadratic penalty factor
                                     seed_number=0)

In [43]:
print('Example of deterministic parameters:', params_t['cs'][:2])

Example of deterministic parameters: tensor([147., 109.])


In [44]:
cost_per_item = lambda Z, Y : params_t['c'].to(dev)*Z.to(dev) \
                            + params_t['cs'].to(dev)*torch.max(torch.zeros((n_items)).to(dev),Y.to(dev)-Z.to(dev)) \
                            + params_t['cw'].to(dev)*torch.max(torch.zeros((n_items)).to(dev),Z.to(dev)-Y.to(dev))


def reshape_outcomes(y_pred, y):
    y_pred = torch.reshape(y_pred, (y_pred.shape[0]//n_items, n_items))
    y = torch.reshape(y, (y.shape[0]//n_items, n_items))
    return y_pred, y

def calc_f_por_item(y_pred, y):
    y_pred, y = reshape_outcomes(y_pred, y)
    z_star =  argmin_solver(y_pred)
    f_per_item = cost_per_item(z_star, y)
    return f_per_item

def calc_f_per_day(y_pred, y):
    f_per_item = calc_f_por_item(y_pred, y)
    f = torch.sum(f_per_item, 1)
    return f

def cost_fn(y_pred, y):
    f = calc_f_per_day(y_pred, y)
    f_total = torch.mean(f)
    return f_total

# Analytical solution to find the argmin
# This function allows autograd (backpropagation)
def argmin_solver(y_pred):
    z_star = y_pred
    return z_star

In [45]:
def train_one_epoch(X_train, y_train, loss_function, optimizer, model):
    
    batches_idx = generate_batches(data_train)
    
    for b in batches_idx:
        x_tr = X_train[b]
        y_tr = y_train[b]

        optimizer.zero_grad()
        #preds = model(x_tr)

        train_loss = loss_function(x_tr, y_tr.reshape(-1))
        train_loss.backward()
        optimizer.step()
        
        
def validate_one_epoch(X, y, val_function, data):
    
    batches_idx = generate_batches(data)
    f_sum = 0
    
    with torch.no_grad():
        for b in batches_idx:
            x_ = X[b]
            y_ = y[b]
    
            f_ = val_function(x_, y_.reshape(-1))  
            f_sum = f_sum + f_

    return f_sum/len(batches_idx)


def validate_one_epoch_final_cost(X, y, model, data):
    
    batches_idx = generate_batches(data)
    f_sum = 0
    
    with torch.no_grad():
        for b in batches_idx:
            x_ = X[b]
            y_ = y[b]
    
            f_ = cost_fn(model(x_).reshape(-1), y_.reshape(-1))  
            f_sum = f_sum + f_

    return f_sum/len(batches_idx)

In [46]:
class Gaussian(object):
    def __init__(self, mu, rho):
        super().__init__()
        self.mu = mu
        self.rho = rho
        self.normal = torch.distributions.Normal(0,1)
    
    @property
    def sigma(self):
        return torch.log1p(torch.exp(self.rho))
    
    def sample(self):
        epsilon = self.normal.sample(self.rho.size()).to(dev)
        return self.mu + self.sigma * epsilon
    
    def log_prob(self, input):
        return (-math.log(math.sqrt(2 * math.pi))
                - torch.log(self.sigma)
                - ((input - self.mu) ** 2) / (2 * self.sigma ** 2)).sum()

class ScaleMixtureGaussian(object):
    def __init__(self, pi, sigma1, sigma2):
        super().__init__()
        self.pi = pi
        self.sigma1 = sigma1
        self.sigma2 = sigma2
        self.gaussian1 = torch.distributions.Normal(0,sigma1)
        self.gaussian2 = torch.distributions.Normal(0,sigma2)
    
    def log_prob(self, input):
        prob1 = torch.exp(self.gaussian1.log_prob(input))
        prob2 = torch.exp(self.gaussian2.log_prob(input))
        return (torch.log(self.pi * prob1 + (1-self.pi) * prob2)).sum()

In [62]:
PI = 0.5
SIGMA_1 = torch.FloatTensor([math.exp(-0)], device=dev)
SIGMA_2 = torch.FloatTensor([math.exp(-6)], device=dev)
n_samples = 3

In [63]:
class SolveNewsvendorWithKKT():
    def __init__(self, params_t, n_samples):
        super(SolveNewsvendorWithKKT, self).__init__()
            
        n_items = len(params_t['c'])
        self.n_items = n_items  
        self.n_samples = n_samples
            
        # Torch parameters for KKT         
        ident = torch.eye(n_items)
        ident_samples = torch.eye(n_items*n_samples)
        ident3 = torch.eye(n_items + 2*n_items*n_samples)
        zeros_matrix = torch.zeros((n_items*n_samples, n_items*n_samples))
        zeros_array = torch.zeros(n_items*n_samples)
        ones_array = torch.ones(n_items*n_samples)
             
        self.Q = torch.diag(
            torch.hstack(
                (
                    params_t['q'], 
                    (1/n_samples)*params_t['qs'].repeat(n_samples), 
                    (1/n_samples)*params_t['qw'].repeat(n_samples)
                )
            )).to(dev)
        
        
        self.lin = torch.hstack(
                                (
                                    params_t['c'], 
                                    (1/n_samples)*params_t['cs'].repeat(n_samples), 
                                    (1/n_samples)*params_t['cw'].repeat(n_samples)
                                )).to(dev)
             
            
        shortage_ineq = torch.hstack(
            (
                -ident.repeat_interleave(n_samples, 0), 
                -ident_samples, 
                zeros_matrix
            )
        )  
        
        
        excess_ineq = torch.hstack(
            (
                ident.repeat_interleave(n_samples, 0), 
                zeros_matrix, 
                -ident_samples
            )
        )
        
        
        price_ineq = torch.hstack(
            (
                params_t['pr'], 
                zeros_array, 
                zeros_array
            )
        )
        
        
        positive_ineq = -ident3
        
        
        self.ineqs = torch.vstack(
            (
                shortage_ineq, 
                excess_ineq, 
                price_ineq, 
                positive_ineq
            )
        ).to(dev)

        
        self.uncert_bound = torch.hstack((-ones_array, ones_array)).to(dev)
        
        self.determ_bound = torch.tensor([params_t['B']]) 
        
        self.determ_bound = torch.hstack((self.determ_bound, 
                                          torch.zeros(n_items), 
                                          torch.zeros(n_items*n_samples), 
                                          torch.zeros(n_items*n_samples))).to(dev)
        
        
        
    def forward(self, y):
        """
        Applies the qpth solver for all batches and allows backpropagation.
        Formulation based on Priya L. Donti, Brandon Amos, J. Zico Kolter (2017).
        Note: The quadratic terms (Q) are used as auxiliar terms only to allow the backpropagation through the 
        qpth library from Amos and Kolter. 
        We will set them as a small percentage of the linear terms (Wilder, Ewing, Dilkina, Tambe, 2019)
        """
        
        n_batches, n_items = y.size()
        
        assert self.n_items == n_items 

        Q = self.Q
        Q = Q.expand(n_batches, Q.size(0), Q.size(1))

        ineqs = torch.unsqueeze(self.ineqs, dim=0)
        ineqs = ineqs.expand(n_batches, ineqs.shape[1], ineqs.shape[2])       

        uncert_bound = (self.uncert_bound*torch.hstack((y, y)))
        determ_bound = self.determ_bound.unsqueeze(dim=0).expand(
            n_batches, self.determ_bound.shape[0])
        bound = torch.hstack((uncert_bound, determ_bound))     
        
        e = torch.DoubleTensor().to(dev)
        
        argmin = QPFunction(verbose=-1)\
            (Q.double(), self.lin.double(), ineqs.double(), 
             bound.double(), e, e).double()
            
        return argmin[:,:n_items]

In [64]:
class BayesianLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        # Weight parameters
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-0.2, 0.2))
        self.weight_rho = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-5,-4))
        self.weight = Gaussian(self.weight_mu, self.weight_rho)
        # Bias parameters
        self.bias_mu = nn.Parameter(torch.Tensor(out_features).uniform_(-0.2, 0.2))
        self.bias_rho = nn.Parameter(torch.Tensor(out_features).uniform_(-5,-4))
        self.bias = Gaussian(self.bias_mu, self.bias_rho)
        # Prior distributions
        self.weight_prior = ScaleMixtureGaussian(PI, SIGMA_1, SIGMA_2)
        self.bias_prior = ScaleMixtureGaussian(PI, SIGMA_1, SIGMA_2)
        self.log_prior = 0
        self.log_variational_posterior = 0

    def forward(self, input, sample=False, calculate_log_probs=False):
        if self.training or sample:
            weight = self.weight.sample()
            bias = self.bias.sample()
        else:
            weight = self.weight.mu
            bias = self.bias.mu
        if self.training or calculate_log_probs:
            self.log_prior = self.weight_prior.log_prob(weight) + self.bias_prior.log_prob(bias)
            self.log_variational_posterior = self.weight.log_prob(weight) + self.bias.log_prob(bias)
        else:
            self.log_prior, self.log_variational_posterior = 0, 0

        return F.linear(input, weight, bias)

In [89]:
class BayesianNetwork(nn.Module):
    def __init__(self, n_feat):
        super().__init__()
        self.act1 = nn.ReLU()
        self.l1 = BayesianLinear(n_feat, 400)
        self.l2 = BayesianLinear(400, 400)
        self.l3 = BayesianLinear(400, 1)
    
    def forward(self, x, sample=False):
        x = self.act1(self.l1(x, sample))
        x = self.act1(self.l2(x, sample))
        x = self.l3(x, sample)
        return x
    
    def forward_uncertain(self, X, n_samples):
        y = torch.zeros(n_samples, X.shape[0]).to(dev)
        for i in range(n_samples):
            y[i] = self(X, sample=True).reshape(-1)
        return y
    
    def log_prior(self):
        return self.l1.log_prior \
               + self.l2.log_prior \
               + self.l3.log_prior
    
    def log_variational_posterior(self):
        return self.l1.log_variational_posterior \
               + self.l2.log_variational_posterior \
               + self.l3.log_variational_posterior
    
    def sample_elbo(self, input, target, n_samples=n_samples):
        log_priors = torch.zeros(n_samples).to(dev)
        log_variational_posteriors = torch.zeros(n_samples).to(dev)
        outputs = self.forward_uncertain(input, n_samples)
        for i in range(n_samples):
            log_priors[i] = self.log_prior()
            log_variational_posteriors[i] = self.log_variational_posterior()
        log_prior = log_priors.mean()
        log_variational_posterior = log_variational_posteriors.mean()
        mse = F.mse_loss(outputs.mean(0), target, size_average=False)
        loss = (log_variational_posterior - log_prior)/NUM_BATCHES + 1*mse
        return loss#, log_prior, log_variational_posterior, mse

In [90]:
# Construct the solver
newsvendor_solve_kkt = SolveNewsvendorWithKKT(params_t, n_samples=n_samples)

In [91]:
h_unc = BayesianNetwork(n_feat=dx)
opt_h_unc = torch.optim.Adam(h_unc.parameters(), lr=0.001)

In [92]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt.forward(y_pred)
    return z_star

In [93]:
n_epochs = 50

rmse_costs_sep = []

for i in range(0, n_epochs):
    train_one_epoch(X_train, y_train, h_unc.sample_elbo, opt_h_unc, h_unc)
    
    with torch.no_grad():
        loss_train = validate_one_epoch(X_train, y_train, h_unc.sample_elbo, data_train)
        loss_val = validate_one_epoch(X_val, y_val, h_unc.sample_elbo, data_val)
        
        f_train = validate_one_epoch_final_cost(X_train, y_train, h_unc, data_train)
        f_val = validate_one_epoch_final_cost(X_val, y_val, h_unc, data_val)

        #rmse_costs_sep.append(val_rmse_sep.data.item())

        print(
              'UNCERTAIN: Loss: ', 
               'Train:', round(loss_train.data.item(), 2),
               '\tVal: ', round(loss_val.data.item(), 2),
               'Cost: ', 
               'Train:', round(f_train.data.item(), 2),
               '\tVal: ', round(f_val.data.item(), 2),
        )

RuntimeError: The size of tensor a (24) must match the size of tensor b (8) at non-singleton dimension 1

In [94]:
h_unc.forward_uncertain(X_test, n_samples=4).shape

torch.Size([4, 396])

In [87]:
h_unc(X_test).shape

torch.Size([396, 1])